In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Data analysis and preparation
------

In [3]:
from zipfile import ZipFile

zip_file_path = "/Users/ALONA/Desktop/Go_IT/HW/Data Science/Data_Science/CLASSIFICATION/Сustomer_transactions/archive.zip"
with ZipFile(zip_file_path, "r") as z:
    for file_info in z.infolist():
        print(file_info.filename)

project1_df.csv


In [5]:
csv_file_name = "project1_df.csv"

with ZipFile(zip_file_path, "r") as z:
    with z.open(csv_file_name) as f:
        df = pd.read_csv(f)

print(df.info())

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55000 entries, 0 to 54999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CID                    55000 non-null  int64  
 1   TID                    55000 non-null  int64  
 2   Gender                 55000 non-null  object 
 3   Age Group              55000 non-null  object 
 4   Purchase Date          55000 non-null  object 
 5   Product Category       55000 non-null  object 
 6   Discount Availed       55000 non-null  object 
 7   Discount Name          27415 non-null  object 
 8   Discount Amount (INR)  55000 non-null  float64
 9   Gross Amount           55000 non-null  float64
 10  Net Amount             55000 non-null  float64
 11  Purchase Method        55000 non-null  object 
 12  Location               55000 non-null  object 
dtypes: float64(3), int64(2), object(8)
memory usage: 5.5+ MB
None


,CID,TID,Gender,Age Group,Purchase Date,Product Category,Discount Availed,Discount Name,Discount Amount (INR),Gross Amount,Net Amount,Purchase Method,Location
0,943146,5876328741,Female,25-45,30/08/2023 20:27:08,Electronics,Yes,FESTIVE50,64.30,725.304000,661.004000,Credit Card,Ahmedabad
1,180079,1018503182,Male,25-45,23/02/2024 09:33:46,Electronics,Yes,SEASONALOFFER21,175.19,4638.991875,4463.801875,Credit Card,Bangalore
2,337580,3814082218,Other,60 and above,06/03/2022 09:09:50,Clothing,Yes,SEASONALOFFER21,211.54,1986.372575,1774.832575,Credit Card,Delhi
3,180333,1395204173,Other,60 and above,04/11/2020 04:41:57,Sports & Fitness,No,NaN,0.00,5695.612650,5695.612650,Debit Card,Delhi
4,447553,8009390577,Male,18-25,31/05/2022 17:00:32,Sports & Fitness,Yes,WELCOME5,439.92,2292.651500,1852.731500,Credit Card,Delhi


### Work with duplicates
-------------------------

In [8]:
duplicates_exist = df.duplicated().any()
print(f"Are there duplicates in the DataFrame: {duplicates_exist}")

num_duplicates = df.duplicated().sum()
print(f"Number of duplicates: {num_duplicates}")

Are there duplicates in the DataFrame: False
Number of duplicates: 0


### Work with NaN
-----------------

In [9]:
df.isnull().sum()

CID                          0
TID                          0
Gender                       0
Age Group                    0
Purchase Date                0
Product Category             0
Discount Availed             0
Discount Name            27585
Discount Amount (INR)        0
Gross Amount                 0
Net Amount                   0
Purchase Method              0
Location                     0
dtype: int64

In [10]:
df['Discount Name'] = df['Discount Name'].fillna('Unknown')

### Determining class imbalance
--------------------------------

In [ ]:
from scipy.stats import entropy

def create_imbalance_pie_chart(df):
    df_columns = df.select_dtypes(include=['object', 'category']).columns
    n_cols = len(df_columns)

    if n_cols == 0:
        print("No categorical columns found in the DataFrame.")
        return
    
    n_rows = (n_cols + 1) // 2  # calculate number of rows needed

    fig, axs = plt.subplots(n_rows, 2, figsize=(12, 5*n_rows))
    axs = axs.flatten() # flatten the 2D array of axes

    for i, column in enumerate(df_columns):
        class_counts = df[column].value_counts().sort_values(ascending=False)

        # calculate the imbalance ratio
        imbalance_ratio = class_counts.iloc[0] / class_counts.iloc[-1]

        axs[i].pie(class_counts.values, labels=class_counts.index, autopct='%1.1f%%', startangle=90)
        axs[i].set_title(f"{column}\nImbalance ratio: {imbalance_ratio:.2f}")

    # remove any unused subplots
    for i in range(n_cols, len(axs)):
        fig.delaxes(axs[i])

    plt.tight_layout()
    plt.show()


def analyze_imbalance(df, column):
    print("Analysis for column: {column}")

    value_count = df[column].value_counts()
    total_count = len(df[column])

    print()

In [14]:
cat_columns = ['Gender', 'Age Group', 'Purchase Date',
               'Product Category', 'Discount Availed',
               'Discount Name', 'Purchase Method', 'Location'
               ]

for col in cat_columns:
    print(f"{col} ->count unique: {len(df[col].unique())}\n{df[col].unique()}")


Gender ->count unique: 3
['Female' 'Male' 'Other']
Age Group ->count unique: 5
['25-45' '60 and above' '18-25' '45-60' 'under 18']
Purchase Date ->count unique: 54988
['30/08/2023 20:27:08' '23/02/2024 09:33:46' '06/03/2022 09:09:50' ...
 '02/08/2024 09:30:44' '05/08/2020 23:57:56' '21/07/2022 09:05:18']
Product Category ->count unique: 9
['Electronics' 'Clothing' 'Sports & Fitness' 'Pet Care' 'Home & Kitchen'
 'Books' 'Beauty and Health' 'Other' 'Toys & Games']
Discount Availed ->count unique: 2
['Yes' 'No']
Discount Name ->count unique: 6
['FESTIVE50' 'SEASONALOFFER21' 'Unknown' 'WELCOME5' 'SAVE10' 'NEWYEARS']
Purchase Method ->count unique: 8
['Credit Card' 'Debit Card' 'PhonePe UPI' 'Google Pay UPI' 'Net Banking'
 'Cash on Delivery' 'Paytm UPI' 'International Card']
Location ->count unique: 14
['Ahmedabad' 'Bangalore' 'Delhi' 'Other' 'Chennai' 'Dehradun' 'Pune'
 'Hyderabad' 'Mumbai' 'Jaipur' 'Lucknow' 'Kolkata' 'Srinagar' 'Varanasi']
